## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [9]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,-46.60,168.33,57.79,70,14,15.41,few clouds
1,1,Jamestown,US,42.10,-79.24,76.64,73,100,0.00,overcast clouds
2,2,Lucea,JM,18.45,-78.17,75.85,77,100,3.60,overcast clouds
3,3,Katsuura,JP,35.13,140.30,80.92,79,100,14.47,overcast clouds
4,4,Ushuaia,AR,-54.80,-68.30,37.06,87,75,3.44,broken clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Jamestown,US,42.10,-79.24,76.64,73,100,0.00,overcast clouds
2,2,Lucea,JM,18.45,-78.17,75.85,77,100,3.60,overcast clouds
3,3,Katsuura,JP,35.13,140.30,80.92,79,100,14.47,overcast clouds
7,7,Kollam,IN,8.88,76.60,76.10,93,100,24.27,overcast clouds
12,12,Trinidad,BO,-14.83,-64.90,81.28,55,87,3.51,overcast clouds
13,13,Padang,ID,-0.95,100.35,78.85,86,96,4.61,overcast clouds
16,16,Bethel,US,41.37,-73.41,75.76,82,0,0.00,clear sky
18,18,Majholi,IN,23.50,79.92,81.64,71,100,3.51,overcast clouds
19,19,Manggar,ID,-2.88,108.27,82.31,75,91,8.72,light rain
20,20,Along,IN,28.17,94.77,82.38,85,99,0.49,overcast clouds


In [15]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                257
City                   257
Country                257
Lat                    257
Lng                    257
Max Temp               257
Humidity               257
Cloudiness             257
Wind Speed             257
Current Description    257
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Jamestown,US,42.10,-79.24,76.64,73,100,0.00,overcast clouds
2,2,Lucea,JM,18.45,-78.17,75.85,77,100,3.60,overcast clouds
3,3,Katsuura,JP,35.13,140.30,80.92,79,100,14.47,overcast clouds
7,7,Kollam,IN,8.88,76.60,76.10,93,100,24.27,overcast clouds
12,12,Trinidad,BO,-14.83,-64.90,81.28,55,87,3.51,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
658,658,Lapi,PH,17.72,121.90,81.01,83,100,1.25,overcast clouds
664,664,Sabang,ID,5.89,95.32,82.29,60,100,2.89,light rain
669,669,Dumas,US,35.87,-101.97,81.61,69,0,6.91,clear sky
673,673,Beaufort,MY,5.35,115.75,88.00,62,93,3.36,overcast clouds


In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Jamestown,US,76.64,overcast clouds,42.10,-79.24,
2,Lucea,JM,75.85,overcast clouds,18.45,-78.17,
3,Katsuura,JP,80.92,overcast clouds,35.13,140.30,
7,Kollam,IN,76.10,overcast clouds,8.88,76.60,
12,Trinidad,BO,81.28,overcast clouds,-14.83,-64.90,
13,Padang,ID,78.85,overcast clouds,-0.95,100.35,
16,Bethel,US,75.76,clear sky,41.37,-73.41,
18,Majholi,IN,81.64,overcast clouds,23.50,79.92,
19,Manggar,ID,82.31,light rain,-2.88,108.27,
20,Along,IN,82.38,overcast clouds,28.17,94.77,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [27]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Jamestown,US,76.64,overcast clouds,42.10,-79.24,Hampton Inn & Suites Jamestown
2,Lucea,JM,75.85,overcast clouds,18.45,-78.17,Grand Palladium Jamaica Resort & Spa
3,Katsuura,JP,80.92,overcast clouds,35.13,140.30,Katsuura Hotel Mikazuki
7,Kollam,IN,76.10,overcast clouds,8.88,76.60,The Raviz Ashtamudi
12,Trinidad,BO,81.28,overcast clouds,-14.83,-64.90,Hotel Piesta
...,...,...,...,...,...,...,...
658,Lapi,PH,81.01,overcast clouds,17.72,121.90,
664,Sabang,ID,82.29,light rain,5.89,95.32,Hotel Citra
669,Dumas,US,81.61,clear sky,35.87,-101.97,"Holiday Inn Express & Suites Dumas, an IHG Hotel"
673,Beaufort,MY,88.00,overcast clouds,5.35,115.75,The Gem Hotel Beaufort


In [28]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_Database.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))